### Making Data Ready

In [1]:
import random
import copy 
import pandas as pd # type: ignore
import numpy as np # type: ignore
import geopandas as gpd # type: ignore
import folium # type: ignore

from shapely.wkt import loads # type: ignore
from math import  sin, cos, sqrt, atan2, radians
from shapely.geometry import Point # type: ignore

random.seed(100)


In [2]:
ontario_map = "../data/maps/ontario.geojson"
file_path_highway = "../data/raw/shapefile_data.csv"
file_path_gas = "../data/raw/ontario_gas_stations.csv"
file_path_pop = "../data/raw/canada_data.xlsx"

ontario_map_gdf = gpd.read_file(ontario_map).to_crs(epsg=3857)
dummy_map = ontario_map_gdf.explode(ignore_index=True).iloc[[2]]
dummy_map['Boundary'] = 'Prince-Edward-County'

# Load highway data
df_highway = pd.read_csv(file_path_highway)
df_highway["geometry"] = df_highway["geometry"].apply(loads)
gdf_full = gpd.GeoDataFrame(df_highway, geometry="geometry")

# Load gas station data
df_gas_station = pd.read_csv(file_path_gas)

# Load population data
df_pop = pd.read_excel(file_path_pop, engine="openpyxl")
df_pop['geometry'] = df_pop.apply(lambda row: Point(row['long'], row['lat']), axis=1)
gdf_pop = gpd.GeoDataFrame(df_pop, geometry="geometry")

In [3]:
# Filter for Prince Edward County
gdf = gdf_full[(gdf_full["CSDNAME_L"] == "Prince Edward County") | (gdf_full["CSDNAME_R"] == "Prince Edward County")]
gdf = gdf.set_crs(epsg=3347, allow_override=True).to_crs(epsg=4326)

In [4]:
# Define bounding box for Prince Edward County
prince_edward_bounds = {
    "min_lat": 43.85,
    "max_lat": 44.10,
    "min_lon": -77.65,
    "max_lon": -76.90
}

# Filter gas stations within the bounding box
df_prince_edward = df_gas_station[
    (df_gas_station["Latitude"] >= prince_edward_bounds["min_lat"]) &
    (df_gas_station["Latitude"] <= prince_edward_bounds["max_lat"]) &
    (df_gas_station["Longitude"] >= prince_edward_bounds["min_lon"]) &
    (df_gas_station["Longitude"] <= prince_edward_bounds["max_lon"])
]

In [5]:
def monte_carlo_sampling(polygon, num_points=10):
    """
    Generates a specified number of random points within a given polygon.

    This function samples random points by generating coordinates within the 
    bounding box of the polygon and checking if they fall inside the polygon. 
    It continues generating points until the required number is reached.

    Parameters:
    - polygon (shapely.geometry.Polygon): The polygon within which points should be sampled.
    - num_points (int, optional): The number of random points to generate. Default is 100.

    Returns:
    - list of shapely.geometry.Point: A list of points inside the polygon.

    Note:
    - This method uses a rejection sampling approach, which may be inefficient 
      for complex or irregularly shaped polygons with large holes.
    """
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < num_points:
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(random_point):
            points.append(random_point)
    return points

In [6]:
def plot_sampled_points(sampled_points, base_map, map_object):
    """
    Plots sampled points on a Folium map as a separate layer.

    This function takes a list of sampled points, converts them into a GeoDataFrame, 
    ensures they have the correct CRS (EPSG:4326) for mapping, and adds them to 
    the provided Folium map as a `FeatureGroup`.

    Parameters:
    - sampled_points (list of shapely.geometry.Point): List of sampled points to be plotted.
    - base_map (geopandas.GeoDataFrame): The base map containing spatial reference (CRS).
    - map_object (folium.Map): The Folium map to which the points will be added.

    Returns:
    - folium.Map: The updated map with the sampled points layer.

    Note:
    - The function automatically reprojects the points and the base map to EPSG:4326 if needed.
    - The added layer can be toggled using the LayerControl feature.
    """

    # Convert points to GeoDataFrame
    points_gdf = gpd.GeoDataFrame(geometry=sampled_points, crs=base_map.crs)

    # Ensure CRS is EPSG:4326 for mapping
    if points_gdf.crs.to_string() != "EPSG:4326":
        points_gdf = points_gdf.to_crs(epsg=4326)
    if base_map.crs.to_string() != "EPSG:4326":
        base_map = base_map.to_crs(epsg=4326)

    # Create FeatureGroup for points
    point_layer = folium.FeatureGroup(name="Sampled Points", overlay=True, control=True)

    # Add each point as a CircleMarker
    for _, row in points_gdf.iterrows():
        point = row.geometry
        folium.CircleMarker(
            location=[point.y, point.x],  # (lat, lon)
            radius=3,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(point_layer)

    # Add the points layer to the map
    point_layer.add_to(map_object)

    # Ensure LayerControl is added for toggling layers
    # folium.LayerControl(collapsed=False).add_to(map_object)

    return map_object,point_layer,points_gdf  # Return the updated map

In [7]:
m = folium.Map(location=[44, -77.2], zoom_start=11)

boundary_layer = folium.FeatureGroup(name="Prince-Edward-County Boundary", 
                                     overlay=True, control=True)

folium.GeoJson(
    dummy_map, 
    name="Prince-Edward-County Boundary", 
    style_function=lambda feature: {"color": "black", "weight": 2, "fillOpacity": 0},
    tooltip=folium.GeoJsonTooltip(fields=["Boundary"])
).add_to(boundary_layer)

boundary_layer.add_to(m)

sampled_points = monte_carlo_sampling(dummy_map.geometry.iloc[0], num_points=10)
m,point_layer,points_gdf= plot_sampled_points(sampled_points, dummy_map, m)
# m

In [8]:
df_prince_edward = pd.DataFrame(df_prince_edward)

# Create a geometry column from the Longitude and Latitude columns
df_prince_edward['geometry'] = df_prince_edward.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

# Convert the DataFrame into a GeoDataFrame
df_prince_edward = gpd.GeoDataFrame(df_prince_edward, geometry='geometry', crs="EPSG:4326")
geometry_gdf = gpd.GeoDataFrame(df_prince_edward[['geometry']], geometry='geometry', crs="EPSG:4326")

geometry_gdf

,geometry
1260,POINT (-77.00704 44.07572)
1559,POINT (-77.58957 44.09635)
1560,POINT (-77.60212 44.09391)
1677,POINT (-77.57969 44.0482)
1907,POINT (-77.60006 44.06114)
2250,POINT (-77.14352 44.00616)
2400,POINT (-77.13588 44.02123)
2408,POINT (-77.16016 43.99984)
2423,POINT (-77.14679 44.00475)
2486,POINT (-77.13918 44.00934)


In [9]:
if dummy_map.crs.to_string() != "EPSG:4326":
    dummy_map = dummy_map.to_crs(epsg=4326)

In [10]:
polygon = dummy_map.geometry.iloc[0]
within_polygon = geometry_gdf.geometry.within(polygon)
geometry_gdf_within = geometry_gdf[within_polygon]

# points_gdf = gpd.GeoDataFrame(geometry=sampled_points, crs=dummy_map.crs)

In [11]:
# if geometry_gdf_within.crs.to_string() != "EPSG:4326":
#     points_gdf.geometry = geometry_gdf_within.to_crs(epsg=4326)

In [12]:
type(geometry_gdf_within)

geopandas.geodataframe.GeoDataFrame

In [13]:
gas_layer = folium.FeatureGroup(name="Gas-Stations", 
                                     overlay=True, control=True)

for idx, row in geometry_gdf_within.iterrows():
    point = row.geometry
    folium.Marker(
        location=[point.y, point.x],  # (lat, lon)
        popup="Fuel"
    ).add_to(gas_layer)

gas_layer.add_to(m)


highway_layer = folium.FeatureGroup(name="Highway-Layer", 
                                     overlay=True, control=True)

geometry_highway = gpd.GeoDataFrame(gdf[['geometry']], geometry='geometry', crs="EPSG:4326")

for idx, row in geometry_highway.iterrows():
    folium.PolyLine(
        locations=[(lat, lon) for lon, lat in row['geometry'].coords],
        color="blue",  
        weight=2.5,    
        opacity=1.0).add_to(highway_layer)

highway_layer.add_to(m)



pop_layer = folium.FeatureGroup(name="Population-Layer",
                               overlay=True, control=True)

cities_to_add = ["Picton", "Wellington"]

for _, row in df_pop.iterrows():
    if row['City'] in cities_to_add:
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=f"{row['City']}: Density {row['Population Density']}",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(pop_layer)

pop_layer.add_to(m)


# folium.LayerControl(collapsed=False).add_to(m)

# m

In [14]:
for key in m._children:
    print(key)

openstreetmap
feature_group_b33180392d096ad16c4dc9bb042755f2
feature_group_79bf59f26678beca7a9724d31fb81472
feature_group_450e899e09a7dae182764c295705d21e
feature_group_d97981ce2b7865d709a693b42083c7ed
feature_group_b16988a3e3a9bfe4c190c5528d592df1


## Changes made specifically for PEI

In [15]:
filtered_gdf_pop = gdf_pop[gdf_pop['City'].isin(cities_to_add)][['City', 'Population', 'Population Density','Area Size','geometry']]
filtered_gdf_pop['Area'] = filtered_gdf_pop['Population'] / filtered_gdf_pop['Population Density']


In [16]:
moving_points = gpd.GeoDataFrame(
    data={
        "gas_station": geometry_gdf_within.geometry.reset_index(drop=True),
        "gen_points": points_gdf.geometry.reset_index(drop=True),
        # "highway": geometry_highway.geometry.reset_index(drop=True),
        "pop": filtered_gdf_pop.geometry.reset_index(drop=True)
        
    } # This will reset the index automatically
)

moving_points.head(8)

,gas_station,gen_points,pop
0,POINT (-77.57969 44.0482),POINT (-77.0725 43.98346),POINT (-77.1333 44)
1,POINT (-77.14352 44.00616),POINT (-77.01686 44.01658),POINT (-77.3534 43.9579)
2,POINT (-77.13588 44.02123),POINT (-77.41792 44.09487),None
3,POINT (-77.16016 43.99984),POINT (-77.51668 43.95462),None
4,POINT (-77.14679 44.00475),POINT (-77.21683 44.13915),None
5,POINT (-77.13918 44.00934),POINT (-77.14069 43.88023),None
6,POINT (-77.24916 43.93899),POINT (-77.03854 44.02766),None
7,POINT (-77.33628 43.95296),POINT (-77.30447 44.05152),None


## !!Note!!
Complexities involved with adding highway on the same dataset as other points so before we add it we better do some processing or we can just access highway information from a seperate dataframe.

# Calculating Score and Moving

In [17]:
# Constant Variables
ACCEPTEBLE_DISTANCE = 5
FACTOR_STEP = 0.5
GAS_STATION_WEIGHT = 0.5



SCORE_GDF = copy.deepcopy(points_gdf)
SCORE_GDF = SCORE_GDF.rename(columns={'geometry': 'gen_points'})

pop_score_gdf = copy.deepcopy(points_gdf)
gas_score_gdf = copy.deepcopy(points_gdf)


gas_score_gdf = gas_score_gdf.rename(columns={'geometry': 'gen_points'})
pop_score_gdf = pop_score_gdf.rename(columns={'geometry': 'gen_points'})

In [18]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c 


def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1

    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)

    initial_bearing = np.degrees(np.arctan2(x, y))
    compass_bearing = (initial_bearing + 360) % 360  
    return compass_bearing 

In [19]:
def closest_factor_bearing(generated_points,factors):
    if generated_points is None:
        return None
    
    closest_distance = float("inf")
    nearest_station = None
    best_bearing = None
    
    for factor in factors:
        dist = haversine(generated_points.y, 
                         generated_points.x, 
                         factor.y,
                         factor.x)

        if dist < closest_distance:
            closest_distance = dist
            nearest_station = factor
            best_bearing = calculate_bearing(generated_points.y, 
                         generated_points.x, 
                         factor.y,
                         factor.x)
            
    return nearest_station, closest_distance, best_bearing

In [20]:
def factor_proximity_score(generated_points, factor_score_calculation_df, factor, factor_name, ACCEPTEBLE_DISTANCE=5, GLOBAL_SCORE_GDF=SCORE_GDF):
    """
    Parameters:
    generated_points : GeoSeries of shapely.geometry.Point
        Generated Points to be evaluated.
    factor_score_calculation_df : GeoDataFrame
        DataFrame for calculating the factor score.
    factor : GeoSeries of shapely.geometry.Point
        Points representing the factor (e.g., gas stations).
    factor_name : str
        Name of the factor (e.g., "gas_station").
    ACCEPTEBLE_DISTANCE : int, optional
        Acceptable distance within which the score is calculated. Default is 5.
    GLOBAL_SCORE_GDF : GeoDataFrame, optional
        Global DataFrame to store the scores. Default is SCORE_GDF.
    """
    factor_score_calculation_df[f"nearest_{factor_name}_point"], factor_score_calculation_df["distance"], factor_score_calculation_df[f"{factor_name}_bearing"] = zip(
        *generated_points.apply(lambda gp: closest_factor_bearing(gp, factor.tolist()))
    )
    GLOBAL_SCORE_GDF[f"{factor_name}_score"] = ACCEPTEBLE_DISTANCE / (factor_score_calculation_df["distance"] + ACCEPTEBLE_DISTANCE)
    factor_score_calculation_df[f"{factor_name}_score"] = GLOBAL_SCORE_GDF[f"{factor_name}_score"]
    return factor_score_calculation_df    

In [21]:
def calculate_step_distance(step, factor_weight, factor_score, distance):
    return step * factor_weight * factor_score * distance
# calculate_step_distance(FACTOR_STEP, GAS_STATION_WEIGHT, SCORE_GDF['gas_station_score'], gas_score_gdf['distance'])

In [22]:
def move_point(lat, lon, bearing_degrees, step, factor_weight, factor_score, distance):
    distance_step = calculate_step_distance(step, factor_weight, factor_score, distance)
    
    R = 6371  
    bearing = np.radians(bearing_degrees)
    lat1, lon1 = np.radians(lat), np.radians(lon)
    
    lat2 = np.arcsin(np.sin(lat1) * np.cos(distance_step / R) +
                     np.cos(lat1) * np.sin(distance_step / R) * np.cos(bearing))
    
    lon2 = lon1 + np.arctan2(np.sin(bearing) * np.sin(distance_step / R) * np.cos(lat1),
                             np.cos(distance_step / R) - np.sin(lat1) * np.sin(lat2))
    point = Point(np.degrees(lon2), np.degrees(lat2))
    return point

In [23]:
factor_proximity_score(gas_score_gdf.gen_points, gas_score_gdf, moving_points.gas_station, "gas_station", ACCEPTEBLE_DISTANCE)
gas_score_gdf['updated_points'] = gas_score_gdf.apply(
    lambda row: move_point(row.gen_points.y, row.gen_points.x, row.gas_station_bearing, FACTOR_STEP, GAS_STATION_WEIGHT, row['gas_station_score'], row['distance']),
    axis=1
)

AttributeError: 'NoneType' object has no attribute 'y'

In [ ]:
gas_score_gdf

,gen_points,nearest_gas_station_point,distance,gas_station_bearing,gas_station_score,updated_points
0,POINT (-77.0725 43.98346),POINT (-77.1391838 44.0093432),6.060944,298.370279,0.452041,POINT (-77.08003 43.98639)
1,POINT (-77.01686 44.01658),POINT (-77.1358803 44.0212321),9.530907,273.149809,0.344094,POINT (-77.0271 44.01699)
2,POINT (-77.41792 44.09487),POINT (-77.3160673 44.0675633),8.683965,110.427219,0.365391,POINT (-77.40862 44.09238)
3,POINT (-77.51668 43.95462),POINT (-77.5796876 44.0481987),11.561871,334.181342,0.301898,POINT (-77.52143 43.96168)
4,POINT (-77.21683 44.13915),POINT (-77.3160673 44.0675633),11.231286,224.904858,0.308047,POINT (-77.22448 44.13364)


In [ ]:
updated_gdf_gas = gpd.GeoDataFrame(gas_score_gdf[['updated_points']], geometry='updated_points', crs="EPSG:4326") 

updated_point_layer = folium.FeatureGroup(name="Singe_Iteration_Updated_point", 
                                     overlay=True, control=True, show = False)

for idx, row in updated_gdf_gas.iterrows():
    point = row.updated_points
    folium.CircleMarker(
        location=[point.y, point.x],  # (lat, lon)
        radius=3,
        color='green',
        fill = True,
        fill_color='green',
        fill_opacity=0.7,
        popup="Singe-Iteration Updated point"
    ).add_to(updated_point_layer)

updated_point_layer.add_to(m)

In [ ]:
iterations = 5
for i in range(iterations):
    factor_proximity_score(gas_score_gdf.updated_points, gas_score_gdf, moving_points.gas_station, "gas_station", ACCEPTEBLE_DISTANCE)
    gas_score_gdf['updated_points'] = gas_score_gdf.apply(
        lambda row: move_point(row.updated_points.y, row.updated_points.x, row.gas_station_bearing, FACTOR_STEP, GAS_STATION_WEIGHT, row['gas_station_score'], row['distance']),
        axis=1
    )

In [ ]:
gas_score_gdf

,gen_points,nearest_gas_station_point,distance,gas_station_bearing,gas_station_score,updated_points
0,POINT (-77.0725 43.98346),POINT (-77.1391838 44.0093432),3.036524,298.347173,0.622160,POINT (-77.11096 43.9984)
1,POINT (-77.01686 44.01658),POINT (-77.1358803 44.0212321),5.712619,273.116678,0.466739,POINT (-77.07286 44.01879)
2,POINT (-77.41792 44.09487),POINT (-77.3160673 44.0675633),5.024346,110.457093,0.498786,POINT (-77.36764 44.0814)
3,POINT (-77.51668 43.95462),POINT (-77.5796876 44.0481987),7.428287,334.165718,0.402308,POINT (-77.54326 43.99413)
4,POINT (-77.21683 44.13915),POINT (-77.3160673 44.0675633),7.143656,224.879693,0.411738,POINT (-77.25947 44.10842)


# Need careful thoughts on scoring formula 
1. Introduce a decay factor (Good for Road/highway -> Cause points are supposed to be extremely close to highway we could also play with acceptable distance) 
2. Inverse fuction approach (Good for Gas-station/Pop Density -> These factors do have some weights but its distance should not affect our overal score as much as highway) (In use)

Road/highway is the highest weighted factor:
NO ROAD = NO CHARGING STATION

Expression=α⋅wf⋅sf*(p) ⋅(∣∣pf−p∣∣)  -> (For step size how much to move its not a stander value rather a dynamic value that adjusts itself based on this formula)

Only move the point if the score is improving

sf*(p) = (ideal_proximity - current Proximity)

### TO DO:
1. Weight all the factors and move based on that right now its only 1 factor and points are moving on that
2. Incorporate after solving the problem mathmatically. 

In [ ]:
def add_updated_points_layer(m, dataframe, column_name , color="black", radius=3, popup_text="Updated_point"):
    # Dynamically create the column name from the factor_name

    # Create a GeoDataFrame using the dynamic column for the geometry
    updated_gdf = gpd.GeoDataFrame(dataframe[f"{column_name}"], geometry=column_name, crs="EPSG:4326")

    # Create a folium FeatureGroup for the updated points
    updated_point_layer__ = folium.FeatureGroup(name="Multi_Iteration_Updated_Points", overlay=True, control=True)

    # Add a CircleMarker for each point in the GeoDataFrame
    for idx, row in updated_gdf.iterrows():
        point = row[column_name]
        folium.CircleMarker(
            location=[point.y, point.x],  # Note: folium expects (lat, lon)
            radius=radius,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=popup_text
        ).add_to(updated_point_layer__)

    # Add the feature group and layer control to the map
    updated_point_layer__.add_to(m)
    return m

add_updated_points_layer(m, gas_score_gdf, "updated_points", color="black", radius=3, popup_text="Multi-Iteration Updated Points")

folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
# updated_gdf_gas = gpd.GeoDataFrame(gas_score_gdf[['updated_points']], geometry='updated_points', crs="EPSG:4326") 

# updated_point_layer = folium.FeatureGroup(name="Updated_Points", 
#                                      overlay=True, control=True)

# for idx, row in updated_gdf_gas.iterrows():
#     point = row.updated_points
#     folium.CircleMarker(
#         location=[point.y, point.x],  # (lat, lon)
#         radius=3,
#         color='green',
#         fill = True,
#         fill_color='green',
#         fill_opacity=0.7,
#         popup="Updated_point"
#     ).add_to(updated_point_layer)

# updated_point_layer.add_to(m)
# folium.LayerControl(collapsed=False).add_to(m)

# m
